In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

In [ ]:
df = pd.read_csv('WO2020-2022OCT.csv') 

In [ ]:
df.head()

In [ ]:
df.size

In [ ]:
df.shape

In [ ]:
df.columns = df.columns.str.replace(' ', '')

In [ ]:
df.info()

In [ ]:
to_remove = ['C3-SICE','LOG', 'C4-ALKHORAYEF', 'HTMS']
df = df[~(df['zzcontractor'].isin(to_remove))]

In [ ]:
#delete column 

df = df.drop(['statusdate' ,'targcompdate', 'targstartdate', 'reportedby',
       'reportdate', 'actstart', 'schedstart', 'schedfinish','supervisor', 'origrecordid',
         'owner', 'lead', 'wogroup', 'scheduler', 'planner', 'projeng1', 'location2',
       'createdby', 'passfailstatus', "persongroup" , "ownergroup", 'changeby', 'changedate', 'description','zzcontractor'] , axis=1)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df = df.rename(columns={"wonum": "work_number", 
                   "assetnum": "asset_ID",
                   "wopriority": "priority",
                   "actfinish": "date", 
                   "historyflag": "modifications",
                   'zzdepartment': "field_type",
                   "zzcontractor": "contractor"}) 

In [ ]:
df.columns

In [ ]:
df['worktype'].value_counts()

In [ ]:
to_remove = ['PM','LOG', '2020', 'CMP', '2021', 'HSE']
df = df[~(df['worktype'].isin(to_remove))]

In [ ]:
df.shape

In [ ]:
df['worktype'].value_counts()

In [ ]:
df['status'].value_counts()

In [ ]:
df.drop(df[ (df['status'] != 'CLOSE') & (df['status'] != 'COMP') ].index , inplace=True)

In [ ]:
df['status'].value_counts()

In [ ]:
df['date']

In [ ]:
df.shape

In [ ]:
df.size

In [ ]:
df['asset_ID'].isnull().sum()

In [ ]:
df['asset_ID']

In [ ]:
df['asset_ID'] = df.asset_ID.interpolate(method='pad')

In [ ]:
df['asset_ID'].isnull().sum()

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df = df.dropna()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
#Deletes the time and keeps only the date
df['date'] = pd.to_datetime(df['date']).dt.date

In [ ]:
#adding year and month columns
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month

In [ ]:
df['field_type'].unique()

In [ ]:
df['field_type'].value_counts()

In [ ]:
#Replacing

ELECTRICAL = ['Low Current','Generator','Appliance', 'BMS', 'Electronic'] #Electrical
MECHANICAL = ['HVAC', 'LPG', 'STP', 'Elevator', 'WTP', 'PMG', 'Fire Alarm', 'Chiller',
              'STP Operations', 'Mitsubishi', 'OCL & Depot', 'Safety'] #Mechanical

df['field_type'].replace(
    to_replace=ELECTRICAL,
    value='Electrical',
    inplace=True
)

df['field_type'].replace(
    to_replace=MECHANICAL,
    value='Mechanical',
    inplace=True
)

df['field_type'] = df.field_type.str.replace('CSSD ', '')

df['field_type'].value_counts()

In [ ]:
df['location' ] = df['location' ].str[:13]

In [ ]:
df['location' ]

In [ ]:
sample = pd.read_csv('ML_sample.csv') 

In [ ]:
sample['worktype' ].value_counts()

In [ ]:
X = sample[['siteid','year', 'month', 'worktype','location', 'field_type']]
y = sample[['work_number']]

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y['work_number'] = le.fit_transform(y['work_number'])
le.classes_
X['location'] = le.fit_transform(X['location'])
X['field_type'] = le.fit_transform(X['field_type'])

X['siteid'] = le.fit_transform(X['siteid'])
X['worktype'] = le.fit_transform(X['worktype'])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

In [ ]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
Y_pred = lr.predict(X_test)

In [ ]:
lr.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

regr = RandomForestClassifier(max_depth=2, random_state=0)
regr.fit(X_train, y_train)

In [ ]:
Y_predtion = regr.predict(X_test)

In [ ]:
lr.score(X_test, y_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

In [ ]:
knn.score(X_test, y_test)